# Отбор релевантных данных

На этом этапе мы чистим данные и убираем нерелевантные посты и нерелевантные комментарии. 

Условия релевантности поста следующие:

- в посте встречаются 2 и более ключевых слов
- в посте встречается хотя бы одно ключевое слово и хотя бы одно контекстное

Условие релевантности поста комментария:

- в комментарии встречается хотя бы одно ключевое слово

In [1]:
import pandas as pd
import polars as pl
from tqdm import tqdm

import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import re
from pymorphy3 import MorphAnalyzer
import string

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
morph = MorphAnalyzer()
translator = str.maketrans('', '', string.punctuation)

nltk.download('stopwords')


df = pl.read_parquet('comm_with_posts_final_10_3.parquet').to_pandas()

In [2]:
with open('kw_pol.txt', 'r', encoding='utf-8') as f:
    kw_new = f.read().split('\n')

with open('kw_context.txt', 'r', encoding='utf-8') as f:
    kw_context = f.read().split('\n')

## Обработка текста

- удаление отметок других пользователей
- приведение к нижнему регистру
- удаление табляций и перенос строк
- удаление знаков препинания
- удаление цифр
- удаление стоп-сов
- лемматизация

In [3]:
# Определение русских стоп-слов
stop_words = stopwords.words('russian')

def preprocess_text(text):

    # Удаление упоминаний других пользователей
    if text.startswith('[id'):
        text = re.sub("\[.*?\,", "", text)
        
    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление табуляций и переносов строк
    text = re.sub(r'[\t\n]', ' ', text)

    # Удаление знаков препинания
    text = re.sub(f"[{re.escape(punctuation)}]", " ", text)

    # Удаление цифр
    text = re.sub("\d+", "", text)

    # Токенизация текста
    tokens = word_tokenize(text)

    # Удаление стоп-слов и лемматизация
    tokens = [morph.parse(token)[0].normal_form for token in tokens if token not in stop_words]

    return " ".join(tokens)

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def find_kw(x):
    count = 0
    for i in kw_new:
        if str(i) in x:
            count += 1
    return count


def find_context(x):
    count = 0
    for i in kw_context:
        if str(i) in x:
            count += 1
    return count

# Фильтрация постов

In [ ]:
df = df.with_columns([
    pl.col("post_text").apply(preprocess_text).alias("cleaned_post"), # обрабатываем текст поста
    
    pl.col("cleaned_post").apply(find_kw).alias("kw_count_post"), # считаем количество ключевых слов
    
    pl.col("cleaned_post").apply(find_context).alias("context_count_post") # считаем количество контекстных слов
])

# фильтруем посты
df = df.filter((pl.col('kw_count_post') >= 2) | 
                ((pl.col('kw_count_post') >= 1) & (pl.col('context_count_post') >= 1)))

# Фильтрация комментариев

In [ ]:
df = df.with_columns([
    pl.col("text").apply(preprocess_text).alias("cleaned_comm"), # обрабатываем текст комментария
    
    pl.col("cleaned_comm").apply(find_kw).alias("kw_count") # считаем количество ключевых слов

])

# фильтруем посты
df = df.filter(pl.col('kw_count') >= 1)

/tmp/ipykernel_1706/1965607339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['cleaned_comm'] = df1['text'].apply(preprocess_text)
/tmp/ipykernel_1706/1965607339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['kw_count'] = df1['cleaned_comm'].apply(find_kw)
/tmp/ipykernel_1706/1965607339.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [18]:
df.to_csv('filtered_pol_df.csv')